# Where in London to open a new Luxury Wine Bar

## MAPS only

In [1]:
## Inporting the necessary packages to import datasets and perform analysis

import pandas as pd
import os
import numpy as np
import matplotlib 
%matplotlib inline
import seaborn as sns

import matplotlib.pyplot as pet

In [2]:
# Importing the Geographic Coordinates of each of UK's 8668 as a CSV format file. 
# It is not possible to import the dataset directly from the indicated web page as there are provided several dataset. 
# Also because some latitude and longitude created problems when imported from my github account I performed a local import for this dataset but an online import for the next one.  

os.chdir("C://@@ Default Folder Python Notebooks")


In [3]:
## Importing the dataset with all neighborhoods / wards from London
neighborhoods = pd.read_csv('Wards__December_2016__Boundaries.csv', encoding='ANSI')
## Observing if the new dataset is similar with the DF. 

In [4]:
# This approach allows us to drop multiple columns at a time 
neighborhoods = neighborhoods.drop(columns=['objectid', 'wd16cd', 'wd16nmw', 'lad16cd', 'bng_e', 'bng_n', 'st_areashape', 'st_lengthshape'])


In [5]:
# Rename Variables 
neighborhoods = neighborhoods.rename(columns={'wd16nm':'Neighborhood',
                        'lad16nm': 'Borough', 
                        'long': 'Longitude',
                        'lat': 'Latitude'})


## 4. Analysis: Mapping London Wards (neighborhoods) <a name="analysis"></a>

In [6]:
## Importing the packages for 

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
address = 'London UK'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of London are 51.5073219, -0.1276474.


## 4.1. Create map of all wards in London <a name="map1"></a>


An initial map of all *654 London wards* was created using the given latitude and longitude. 
Using *Fourthsquare API* a radius of 500 meters was chosen to identify venues of interest around indicated wards. 
Finally, a total number of *12192 places of interest* were identified around these 654 wards. 

In [8]:
map_London = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], 
                                           neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_London)  
    
map_London

Map 1. Representation of all 654 London wards. 

In [9]:
## Importing the dataset with all Venues from London as exported by the Fourthsquare API 
London_venues = pd.read_csv('https://raw.githubusercontent.com/sebastianBIanalytics/Coursera_Capstone/master/London_Venues.csv', encoding='ANSI')

In [10]:
## Count the number of venues in each neighborhood
## With a single exception the maximum number of venues was 100.  
Venues_Ward = London_venues.groupby('Neighborhood').count().sort_values('Venue', ascending=False)
#Venues_Ward.head()

In [11]:
# one hot encoding
London_onehot = pd.get_dummies(London_venues[['Venue Category']], prefix="", prefix_sep="")
#London_onehot.head()

In [12]:
# add neighborhood column back to dataframe
London_onehot['Neighborhood'] = London_venues['Neighborhood'] 
#London_onehot.head()

In [13]:
# move neighborhood column to the first column
fixed_columns = [London_onehot.columns[-1]] + list(London_onehot.columns[:-1])
London_onehot = London_onehot[fixed_columns]

#London_onehot.head()

In [14]:
London_onehot.shape

(12192, 426)

In [15]:
London_grouped = London_onehot.groupby('Neighborhood').mean().reset_index()
#London_grouped.head()

In [16]:
## Identify the top 5 venues in each Ward

num_top_venues = 5

for hood in London_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = London_grouped[London_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbey----
           venue  freq
0  Grocery Store  0.12
1            Pub  0.09
2    Coffee Shop  0.09
3    Pizza Place  0.06
4    Supermarket  0.06


----Abbey Road----
                       venue  freq
0                      CafÃ©  0.09
1          French Restaurant  0.05
2              Grocery Store  0.05
3         Persian Restaurant  0.05
4  Middle Eastern Restaurant  0.05


----Abbey Wood----
                           venue  freq
0                  Grocery Store   0.5
1                     Playground   0.5
2                            ATM   0.0
3  Paper / Office Supplies Store   0.0
4                      Pet Store   0.0


----Abingdon----
                 venue  freq
0                Hotel  0.08
1   Italian Restaurant  0.07
2                  Pub  0.07
3                CafÃ©  0.07
4  Japanese Restaurant  0.03


----Acton Central----
             venue  freq
0    Grocery Store  0.25
1    Train Station  0.12
2            Hotel  0.12
3   Breakfast Spot  0.12
4  Bed & Breakfast  

                venue  freq
0      Sandwich Place  0.08
1         Coffee Shop  0.08
2    Sushi Restaurant  0.06
3            Bus Stop  0.06
4  Italian Restaurant  0.05


----Blendon and Penhill----
               venue  freq
0              CafÃ©   0.2
1      Grocery Store   0.2
2        Supermarket   0.2
3  Indian Restaurant   0.2
4                Pub   0.2


----Bloomsbury----
               venue  freq
0        Coffee Shop  0.16
1            Exhibit  0.10
2                Bar  0.06
3          Bookstore  0.04
4  French Restaurant  0.04


----Boleyn----
              venue  freq
0             CafÃ©  0.22
1               Pub  0.22
2  Toy / Game Store  0.11
3     Grocery Store  0.11
4       Pizza Place  0.11


----Botwell----
                    venue  freq
0                    Park   0.6
1  Furniture / Home Store   0.2
2                     Pub   0.2
3                     ATM   0.0
4                  Palace   0.0


----Bounds Green----
                       venue  freq
0               

           venue  freq
0            Pub  0.16
1          CafÃ©  0.11
2      Gastropub  0.11
3       Bus Stop  0.05
4  Grocery Store  0.05


----Canons----
                venue  freq
0         Golf Course   0.5
1                Park   0.5
2                 ATM   0.0
3  Photography Studio   0.0
4           Pet Store   0.0


----Cantelowes----
                 venue  freq
0                  Pub  0.19
1                 Park  0.10
2        Garden Center  0.10
3                CafÃ©  0.10
4  Rental Car Location  0.05


----Carshalton Central----
           venue  freq
0            Pub  0.33
1           Park  0.11
2  Grocery Store  0.11
3            Spa  0.11
4  Historic Site  0.11


----Carshalton South and Clockhouse----
       venue  freq
0    Stables   0.5
1       Park   0.5
2        ATM   0.0
3     Palace   0.0
4  Pet Store   0.0


----Castle Baynard----
                venue  freq
0                 Pub  0.13
1         Coffee Shop  0.11
2  Italian Restaurant  0.07
3      Sandwich Place 

4                         ATM  0.00


----Copers Cope----
                  venue  freq
0             Irish Pub  0.29
1         Train Station  0.14
2  Fast Food Restaurant  0.14
3                  Park  0.14
4          Tennis Court  0.14


----Coppetts----
                  venue  freq
0    Italian Restaurant  0.11
1           Supermarket  0.11
2           Coffee Shop  0.11
3  Gym / Fitness Center  0.11
4           Pizza Place  0.11


----Cordwainer----
                   venue  freq
0            Coffee Shop  0.09
1     Italian Restaurant  0.06
2         Sandwich Place  0.05
3   Gym / Fitness Center  0.05
4  Vietnamese Restaurant  0.03


----Cornhill----
                   venue  freq
0            Coffee Shop  0.07
1         Sandwich Place  0.07
2   Gym / Fitness Center  0.06
3  Vietnamese Restaurant  0.04
4             Steakhouse  0.04


----Coulsdon East----
               venue  freq
0  Fish & Chips Shop  0.29
1               Park  0.14
2         Restaurant  0.14
3               Lak

                 venue  freq
0                  Bar  0.14
1                Hotel  0.14
2        Grocery Store  0.14
3  Japanese Restaurant  0.07
4            Wine Shop  0.07


----East Sheen----
                venue  freq
0               CafÃ©   0.4
1  Athletics & Sports   0.2
2           Racetrack   0.2
3                Lake   0.2
4                 ATM   0.0


----East Walworth----
         venue  freq
0  Coffee Shop  0.12
1          Pub  0.12
2  Pizza Place  0.12
3        CafÃ©  0.12
4        Hotel  0.12


----East Wickham----
           venue  freq
0  Grocery Store  0.25
1            Pub  0.25
2    Coffee Shop  0.25
3  Train Station  0.25
4            ATM  0.00


----Eastbrook----
                        venue  freq
0                        Park   0.4
1  Construction & Landscaping   0.2
2         Sporting Goods Shop   0.2
3                        Lake   0.2
4                         ATM   0.0


----Eastbury----
            venue  freq
0     Auto Garage   0.2
1   Grocery Store   0.2

...

4  Pet CafÃ©   0.0


----Monkhams----
                  venue  freq
0                   Pub  0.67
1  Gym / Fitness Center  0.33
2                   ATM  0.00
3                Palace  0.00
4             Pet Store  0.00


----Mortlake and Barnes Common----
                venue  freq
0  Athletics & Sports  0.12
1          Restaurant  0.12
2                 Pub  0.12
3            Tea Room  0.06
4         Coffee Shop  0.06


----Mottingham and Chislehurst North----
                        venue  freq
0                        Park  0.25
1                         Bar  0.25
2  Construction & Landscaping  0.25
3                    Bus Stop  0.25
4                         ATM  0.00


----Munster----
                venue  freq
0               CafÃ©  0.25
1  Italian Restaurant  0.18
2                 Pub  0.07
3         Pizza Place  0.07
4  Chinese Restaurant  0.04


----Muswell Hill----
                  venue  freq
0                   Pub  0.18
1           Coffee Shop  0.09
2              Bus 

               venue  freq
0                Pub  0.22
1        Coffee Shop  0.08
2              CafÃ©  0.08
3        Art Gallery  0.05
4  Indian Restaurant  0.05


----Queen's Gate----
                venue  freq
0               Hotel  0.19
1         Coffee Shop  0.07
2  Italian Restaurant  0.07
3                 Pub  0.06
4          Restaurant  0.06


----Queen's Park----
                venue  freq
0         Yoga Studio   0.2
1                 Gym   0.2
2               Canal   0.2
3  Italian Restaurant   0.1
4               CafÃ©   0.1


----Queenhithe----
                  venue  freq
0           Coffee Shop  0.11
1    Italian Restaurant  0.06
2            Art Museum  0.04
3        Sandwich Place  0.03
4  Gym / Fitness Center  0.03


----Queens Park----
               venue  freq
0              CafÃ©  0.09
1        Yoga Studio  0.06
2        Coffee Shop  0.06
3  Indian Restaurant  0.06
4               Park  0.06


----Queensbury----
                venue  freq
0   Indian Restaurant 

4                Plaza  0.07


----St Katharine's & Wapping----
                  venue  freq
0                  Park  0.06
1                   Bar  0.06
2  Gym / Fitness Center  0.06
3                   Pub  0.06
4         Grocery Store  0.06


----St Leonard's----
                  venue  freq
0                   Pub  0.10
1         Grocery Store  0.10
2  Gym / Fitness Center  0.06
3              Pharmacy  0.06
4           Coffee Shop  0.06


----St Margarets and North Twickenham----
                venue  freq
0                 Pub  0.27
1                Park  0.09
2                 Gym  0.09
3  Seafood Restaurant  0.09
4       Grocery Store  0.09


----St Mark's----
                venue  freq
0         Coffee Shop  0.13
1                 Pub  0.11
2   Indian Restaurant  0.07
3       Grocery Store  0.04
4  Italian Restaurant  0.04


----St Mary's----
          venue  freq
0           Pub  0.15
1        Bakery  0.04
2       Theater  0.04
3  Burger Joint  0.04
4          Park  0.04



4     Platform  0.04


----Twickenham Riverside----
               venue  freq
0              CafÃ©  0.18
1  French Restaurant  0.12
2          Gastropub  0.12
3      Historic Site  0.06
4               Park  0.06


----Underhill----
                venue  freq
0       Grocery Store   1.0
1                 ATM   0.0
2  Photography Studio   0.0
3           Pet Store   0.0
4           Pet CafÃ©   0.0


----Upminster----
                           venue  freq
0     Construction & Landscaping   0.5
1                      Pet Store   0.5
2                            ATM   0.0
3  Paper / Office Supplies Store   0.0
4             Photography Studio   0.0


----Upper Edmonton----
                  venue  freq
0                   Pub  0.15
1           Supermarket  0.15
2                Bakery  0.08
3         Grocery Store  0.08
4  Fast Food Restaurant  0.08


----Upper Norwood----
                           venue  freq
0             English Restaurant   0.5
1                           Park   0.

4                       Park  0.00


----Woolwich Riverside----
                  venue  freq
0         Train Station  0.25
1                  Park  0.25
2  Fast Food Restaurant  0.25
3                 CafÃ©  0.25
4              Pharmacy  0.00


----Worcester Park----
                  venue  freq
0                  Park  0.50
1             Racetrack  0.25
2  Fast Food Restaurant  0.25
3                   ATM  0.00
4                Palace  0.00


----Wormholt and White City----
                  venue  freq
0         Grocery Store  0.15
1                 CafÃ©  0.15
2                  Park  0.10
3  Fast Food Restaurant  0.10
4             Gastropub  0.10


----Yeading----
                  venue  freq
0    Seafood Restaurant  0.17
1       Harbor / Marina  0.17
2        Hardware Store  0.17
3           Supermarket  0.17
4  Fast Food Restaurant  0.17


----Yiewsley----
                           venue  freq
0     Construction & Landscaping   0.5
1                   Soccer Field   0.5
2  

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
## Identify the top 10 venues in each Ward

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = London_grouped['Neighborhood']

for ind in np.arange(London_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)


## New dataset with top 10 venues sorted per Ward. 
    
# neighborhoods_venues_sorted

## 4.3. Clustering the Wards/ Neighborhoods in London <a name="explore"></a>

In [19]:
# set number of clusters
kclusters = 6

London_grouped_clustering = London_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(London_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 5, 0, 5, 5, 0, 0, 0, 0])

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
## Change name 
London_merged = neighborhoods

# merge Downtown Toronto_grouped with Downtown Toronto_data to add latitude/longitude for each neighborhood
London_merged = London_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), 
                                                       on='Neighborhood')

In [21]:
London_merged = London_merged.dropna()

In [22]:
London_merged['Cluster Labels'] = London_merged['Cluster Labels'].astype(int)
#London_merged

## 4.3.1. London Wards distribution according to venue Clustering <a name="explore2"></a>


In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(London_merged['Latitude'], London_merged['Longitude'], 
                                  London_merged['Neighborhood'], 
                                  London_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

Map 2. Representation of all London Wards as Venue Clusters. 



In [25]:
HighIncome = London_merged["Neighborhood"].isin(['Garden Suburb', 'Belsize', 'Frognal and Fitzjohns', 'Hampstead Town', 'Swiss Cottage', 'Southfield', 'Munster', 
'Palace Riverside', 'Parsons Green and Walham', 'Alexandra', 'Crouch End', 'Highgate', 'Muswell Hill', 
'Chiswick Homefields', 'Abingdon', 'Brompton', 'Campden', 'Courtfield', 'Hans Town', 'Holland', 'Norland', 
'Pembridge', "Queen's Gate", 'Redcliffe', 'Royal Hospital', 'Stanley', 'Hillside', 'Village', 'Wimbledon Park', 
'Barnes', 'East Sheen', 'Kew', 'Mortlake and Barnes Common', 'North Richmond', 'St. Margarets & North Twickenham', 
'South Richmond', 'South Twickenham', 'Teddington', 'Twickenham Riverside', 'Village', "St. Katherine's and Wapping", 
'Balham', 'Nightingale', 'Northcote', 'Shaftesbury', 'Thamesfield', 'Wandsworth Common', 'Abbey Road', 
'Bryanston and Dorset Square', 'Knightsbridge and Belgravia', 'Marylebone High Street', "St. James's", 
'Warwick', 'West End'])

HighIncome_clusters = London_merged[HighIncome]

In [26]:
# create map
map_clustersHighincome = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(HighIncome_clusters['Latitude'], HighIncome_clusters['Longitude'], 
                                  HighIncome_clusters['Neighborhood'], 
                                  HighIncome_clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clustersHighincome)
       
map_clustersHighincome

Map 3. Representation of High Income Wards as Venue Clusters. 

In [27]:
HighIncome_clusters.sort_values('Borough', ascending = False)

,Neighborhood,Borough,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
552,Bryanston and Dorset Square,Westminster,-0.162910,51.520309,0,Middle Eastern Restaurant,Coffee Shop,Pub,Hotel,CafÃ©,Sandwich Place,Pizza Place,Lebanese Restaurant,Ice Cream Shop,Japanese Restaurant
550,Abbey Road,Westminster,-0.179950,51.534088,0,CafÃ©,Recording Studio,Historic Site,Metro Station,Gift Shop,Persian Restaurant,Garden,Grocery Store,Gym / Fitness Center,Boat or Ferry
569,West End,Westminster,-0.145430,51.511341,0,Clothing Store,Boutique,Hotel,Cosmetics Shop,Art Gallery,Lounge,Toy / Game Store,Steakhouse,French Restaurant,Food Court
567,Warwick,Westminster,-0.143920,51.492279,0,Hotel,Pub,Italian Restaurant,Coffee Shop,CafÃ©,Restaurant,Turkish Restaurant,Chinese Restaurant,Bakery,Sandwich Place
561,Marylebone High Street,Westminster,-0.149160,51.519539,0,Hotel,French Restaurant,Burger Joint,Pizza Place,CafÃ©,Indian Restaurant,Italian Restaurant,Juice Bar,Sandwich Place,Sushi Restaurant
557,Knightsbridge and Belgravia,Westminster,-0.168270,51.502411,0,Coffee Shop,CafÃ©,Italian Restaurant,Boutique,Gym / Fitness Center,Chinese Restaurant,Garden,Steakhouse,Outdoor Sculpture,Fountain
545,Thamesfield,Wandsworth,-0.223530,51.466171,4,Pub,Gym / Fitness Center,Japanese Restaurant,CafÃ©,Coffee Shop,Sushi Restaurant,Restaurant,Tennis Court,Gastropub,Bar
543,Shaftesbury,Wandsworth,-0.158930,51.465961,0,Pub,CafÃ©,Indian Restaurant,Portuguese Restaurant,Italian Restaurant,Thai Restaurant,Indonesian Restaurant,Restaurant,Sushi Restaurant,Supermarket
539,Northcote,Wandsworth,-0.163800,51.456310,0,Pub,Bakery,CafÃ©,Italian Restaurant,Pizza Place,Burger Joint,Sporting Goods Shop,Coffee Shop,Restaurant,Antique Shop
538,Nightingale,Wandsworth,-0.158940,51.442940,0,Coffee Shop,Pub,Italian Restaurant,Bookstore,English Restaurant,Sandwich Place,Grocery Store,Bar,Bakery,Train Station


In [28]:
Wine_Bars = London_venues["Venue Category"].isin(['Winery', 'Wine Bar'])

Wine_Bars = London_venues[Wine_Bars]

Wine_Bars


,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
810,810,Bloomsbury,51.521900,-0.13256,Vagabond,51.518695,-0.135003,Wine Bar
1085,1085,Holborn and Covent Garden,51.518978,-0.11466,Noble Rot,51.521880,-0.118596,Wine Bar
1152,1152,Kentish Town,51.551701,-0.13929,Authentique - Epicerie & Bar,51.555434,-0.139411,Wine Bar
1202,1202,Kilburn,51.540352,-0.18741,The Arches,51.542022,-0.180726,Wine Bar
1341,1341,St Pancras and Somers Town,51.534859,-0.13118,The Drop,51.535656,-0.126819,Wine Bar
2492,2492,Parsons Green and Walham,51.473888,-0.19693,Amuse Bouche,51.475183,-0.200816,Wine Bar
2593,2593,Shepherd's Bush Green,51.506512,-0.22563,Albertine,51.505410,-0.224597,Wine Bar
2703,2703,Town,51.475769,-0.20347,Amuse Bouche,51.475183,-0.200816,Wine Bar
4489,4489,Coldharbour,51.464561,-0.10866,The New Zealand Wine Cellar,51.463418,-0.112496,Wine Bar
4506,4506,Coldharbour,51.464561,-0.10866,The Wine Parlour,51.461466,-0.111461,Winery


In [29]:
map_wine_bars = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(Wine_Bars['Venue Latitude'], Wine_Bars['Venue Longitude'], 
                                           Wine_Bars['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_wine_bars)  
    
map_wine_bars

Map 4. Representation of existing Wine Bars as provided by Fourthsquare API.  

            
* If you read this Notebook please email me any comments or suggestions to my email sebastian@bianalytics.org                







